In [20]:
# !chcp 1251
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, ScalarFormatter
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec

In [21]:
import sys
sys.path.append("E:\Soft\pypi\lasio-0.31.tar\\")
import lasio

In [22]:
# Настройка рабочего стола и вывода инфы в пандас
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
pd.options.display.max_columns = 15
pd.options.display.precision = 7
pd.options.display.max_colwidth = 50

In [ ]:
mer_init = pd.read_csv('../all_mers/nng_mur_mer_20230519.csv', encoding = 'cp1251', sep = ';')
rigis_init = pd.read_csv('../all_mers/nng_mur_rigis_20230519.csv', encoding = 'cp1251', sep = ';')
addinfo_init = pd.read_csv('../all_mers/nng_mur_addinfo_20230519.csv', encoding = 'cp1251', sep = ';')
padcoord_init = pd.read_csv('../all_mers/nng_mur_padcoord_20230522.csv', encoding = 'cp1251', sep = ';')
geobd_ngt_init = pd.read_csv('../all_mers/nng_mur_geobd_ngt_20230522.csv', encoding = 'cp1251', sep = ';')

## Data Upload

In [319]:
%%time
data1_init = pd.read_excel('Скв. 6001 (01.01.2000-30.11.2022).xls', header = 2)
data2_init = pd.read_csv('incl.csv', encoding = 'cp1251', sep = ';')
data3_init = pd.read_csv('vyng_ds_download.csv', encoding = 'utf8', sep = ';')

Wall time: 313 ms


In [1]:
%%time
data1_init.rename(columns = 
            {'Unnamed: 0':'date',
            'Тип':'pump_type',
            'Нсп':'depth_pump',
            'ТМ':'tm',
            'Дисп':'disp',
            'OIS':'ois',
            'Qж_мас ТМ':'q_liqm_tm',
            'Qж':'q_liq',
            'Qж ТМ':'q_liq_tm',
            'Qж ТМ (исх)':'q_liq_tm_init'}, inplace=True)
data1 = data1_init[['date', 'q_liq', 'q_oil', 'wc', 'bhp_rsch', 'pump_type']]
data1['date'] = data1['date'].astype("string")
data1['date'] = data1['date'].apply(lambda x: month_change(x))
data1['date'] = pd.to_datetime(data1['date'])
convert_dict = {'oilfield': 'string',
                'well': 'string',
                'zone': 'string',
                'top_dst': 'float64',
                'bot_dst': 'float64',
               } 
well_test = well_test.astype(convert_dict)

## Data Processing

In [3]:
def mer_month_quant(mer, period = 6):
    oil_cumul = pd.DataFrame(mer.groupby(['well'])['oil_t'].apply(lambda x: np.nansum(np.array(x[:period]))))
    work_time_cumul =  pd.DataFrame(mer.groupby(['well'])['work_time'].apply(lambda x: np.nansum(np.array(x[:period]))))
    oil_cumul.rename(columns={'oil_t':'cum_oil_t'}, inplace=True)
    work_time_cumul.rename(columns={'work_time':'work_time_hr'}, inplace=True)
    result = oil_cumul.join(work_time_cumul)
    return result

## Final Result

In [ ]:
data1.well.unique()

In [ ]:
ax.hist(oil_6m.oil_6m, bins=20, color = 'green', alpha=0.5, ec="black")

In [423]:
for num, wellPlot in enumerate(data1.well.unique()):
    f, ax = plt.subplots(1,2,figsize = (15,10))
    wellLog1 = data1.query(f'well == "{wellPlot}"')
    wellLog2 = wellLog1.query('pred_test_ss == 1')
    ax[0].set_title(wellPlot + '_logreg', fontsize=20)
    # ax[0].yaxis.set_major_locator(MultipleLocator(10))
    x = wellLog1.gr
    y = wellLog1.tvdss
    xz = wellLog2.gr
    yz = wellLog2.tvdss
    ax[0].scatter(xz, yz, c = 'red')
#     ax[0].set_xlim(2,16)
    ax[0].invert_yaxis()
    ax[0].grid()

    wellLog_orig1 = data2.query(f'well == "{wellPlot}"')
    wellLog_orig2 = wellLog_orig1.query('zones == "БС9-2-1+2_TOP_S"')
    ax[1].set_title(wellPlot + '_manual', fontsize=20)
    # ax[1].yaxis.set_major_locator(MultipleLocator(10))
    x_orig = wellLog_orig1.gr
    y_orig = wellLog_orig1.tvdss
    xz_orig = wellLog_orig2.gr
    yz_orig = wellLog_orig2.tvdss
    ax[1].scatter(x_orig,y_orig, c = 'green')
    ax[1].scatter(xz_orig, yz_orig, c = 'red')
#     ax[1].set_xlim(2,16)
    ax[1].invert_yaxis()
    ax[1].grid()

In [ ]:
def well_plot_creation(well):
    fig = plt.figure(figsize=(20, 5))
    ws = [1.5, 5]
    hs = [1, 1]
    gs = GridSpec(ncols=2, nrows=2, figure=fig, width_ratios=ws, height_ratios=hs)

    ax1 = plt.subplot(gs[:, 0])
    ax1.set_title(well)
    ax1.step(mer_sel[mer_sel['имя скважины'] == well]['дата(дд.мм.гггг)'], mer_sel[mer_sel['имя скважины'] == well]['вода т/сут'], c='blue', label = 'вода')
    ax1.fill_between(mer_sel[mer_sel['имя скважины'] == well]['дата(дд.мм.гггг)'], mer_sel[mer_sel['имя скважины'] == well]['вода т/сут'], step="pre", alpha=0.4)
    ax1.step(mer_sel[mer_sel['имя скважины'] == well]['дата(дд.мм.гггг)'], mer_sel[mer_sel['имя скважины'] == well]['нефть т/сут'], c='brown', label = 'нефть', linewidth = 4)
    ax1.fill_between(mer_sel[mer_sel['имя скважины'] == well]['дата(дд.мм.гггг)'], mer_sel[mer_sel['имя скважины'] == well]['нефть т/сут'], step="pre", alpha=0.5)
    ax1.xaxis.set_major_formatter(NullFormatter())
    ax1.grid(axis = 'y')
    ax1.legend()

    ax2 = fig.add_subplot(gs[0,1])
    ax2.plot(data_gis[data_gis['wellName'] == well]['MD'], data_gis[data_gis['wellName'] == well]['TVDSS'], '--', c='black', label = 'TVDSS')
    ax2.invert_yaxis()
    ax2.legend(loc='upper right')

    ax2.grid()
    ax2 = ax2.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.plot(data_gis[data_gis['wellName'] == well]['MD'], data_gis[data_gis['wellName'] == well]['GR'], '--', c='green', label = 'GR')
    ax2.set_ylim(0, max(data_gis[data_gis['wellName'] == well]['GR'])*1.1)
    ax2.legend(bbox_to_anchor=(1, 0.85))
    ax3 = fig.add_subplot(gs[1,1])
    ax3.semilogy(data_gis[data_gis['wellName'] == well]['MD'], data_gis[data_gis['wellName'] == well]['RT_SH'], c='red', label = 'RT_SH')
    ax3.semilogy(data_gis[data_gis['wellName'] == well]['MD'], data_gis[data_gis['wellName'] == well]['RT_DP'], label = 'RT_DP')
    ax3.yaxis.set_major_formatter(ScalarFormatter())
    ax3.set_ylim(1, 100)
    ax3.legend()
    ax3.grid(which='major')
    ax3.grid(True, which='minor')
    fig.tight_layout()
    plt.tight_layout()
    return

In [ ]:
for i, name_c in enumerate(frac_mer_final_high.index):
    ax[1].annotate(name_c, (x_coord_high[i]+100, y_coord_high[i]+100), fontsize = 10)

In [ ]:
sf = ScalarFormatter()
sf.set_powerlimits((-4, 4))
ax.yaxis.set_major_formatter(sf)

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.displot(data=sns_rigis, x='rt', bins=30, col='well', kde=True, height=5)
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(2))
plt.xlim(0,30);

In [1]:
# Get the full list of styles
import seaborn as sns
sns.axes_style()
custom = {"axes.edgecolor": "black", "grid.linestyle": "dashed", "grid.color": "gray"}
sns.set_style("whitegrid", rc = custom)

In [ ]:
def zone_oil_sum(mer=mer):
    zonesOilSum = pd.DataFrame(mer.groupby('zone')['oil'].sum())
    zonesOilSum.rename(columns={'oil':'oil_sum'}, inplace=True)
    zonesOilSum = zonesOilSum.sort_values('oil_sum', ascending=False)
    zonesOilSum[['oil_sum']] = zonesOilSum[['oil_sum']].applymap('{:,.0f}'.format)
    return zonesOilSum